<a href="https://colab.research.google.com/github/rajagopalmotivate/DeepLearningforSocietal/blob/main/Copy_of_DCGAN_Mnist_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Model
from keras.optimizers import Adam
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import plot_model
from google.colab import files



Using TensorFlow backend.


In [3]:
def load_data():
    (X_train, _), (_, _) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    return X_train

In [4]:
def build_generator(noise_shape=(100,)):
    input = Input(noise_shape)
    x = Dense(128 * 7 * 7, activation="relu")(input)
    x = Reshape((7, 7, 128))(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = UpSampling2D()(x)
    x = Conv2D(128, kernel_size=3, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = UpSampling2D()(x)
    x = Conv2D(64, kernel_size=3, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Conv2D(1, kernel_size=3, padding="same")(x)
    out = Activation("tanh")(x)
    model = Model(input, out)
    print("-- Generator -- ")
    model.summary()
    return model

In [5]:
def build_discriminator(img_shape):
    input = Input(img_shape)
    x =Conv2D(32, kernel_size=3, strides=2, padding="same")(input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64, kernel_size=3, strides=2, padding="same")(x)
    x = ZeroPadding2D(padding=((0, 1), (0, 1)))(x)
    x = (LeakyReLU(alpha=0.2))(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Conv2D(128, kernel_size=3, strides=2, padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Conv2D(256, kernel_size=3, strides=1, padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    out = Dense(1, activation='sigmoid')(x)

    model = Model(input, out)
    print("-- Discriminator -- ")
    model.summary()
    return model

In [6]:
def train(generator, discriminator, combined, epochs=2000, batch_size=128, save_interval=50):

    X_train = load_data()

    num_examples = X_train.shape[0]
    num_batches = int(num_examples / float(batch_size))
    print('Number of examples: ', num_examples)
    print('Number of Batches: ', num_batches)
    print('Number of epochs: ', epochs)

    half_batch = int(batch_size / 2)

    for epoch in range(epochs + 1):
        for batch in range(num_batches):

            # noise images for the batch
            noise = np.random.normal(0, 1, (half_batch, 100))
            fake_images = generator.predict(noise)
            fake_labels = np.zeros((half_batch, 1))

            # real images for batch
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            real_images = X_train[idx]
            real_labels = np.ones((half_batch, 1))

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = discriminator.train_on_batch(real_images, real_labels)
            d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            noise = np.random.normal(0, 1, (batch_size, 100))
            # Train the generator
            g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))

            # Plot the progress
            print("Epoch %d Batch %d/%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
                  (epoch,batch, num_batches, d_loss[0], 100 * d_loss[1], g_loss))

            if batch % 50 == 0:
                save_imgs(generator, epoch, batch)





In [7]:
def save_imgs(generator, epoch, batch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig("mnist_%d_%d.png" % (epoch, batch))
    files.download("mnist_%d_%d.png" % (epoch, batch))
    plt.show()
    plt.close()
    

In [8]:
def build_models():

    gen_optimizer = Adam(lr=0.0002, beta_1=0.5)
    disc_optimizer = Adam(lr=0.0002, beta_1=0.5)


    discriminator = build_discriminator(img_shape=(28, 28, 1))
    discriminator.compile(loss='binary_crossentropy',
                               optimizer=disc_optimizer,
                               metrics=['accuracy'])

    generator = build_generator()
    generator.compile(loss='binary_crossentropy', optimizer=gen_optimizer)

    z = Input(shape=(100,))
    img = generator(z)
    discriminator.trainable = False
    real = discriminator(img)
    combined = Model(z, real)
    combined.compile(loss='binary_crossentropy', optimizer=gen_optimizer)
    return generator, discriminator, combined

In [9]:
generator, discriminator, combined = build_models()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
-- Discriminator -- 
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_______________________

In [12]:
files.download('gen.png')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
!ls

gen.png		  mnist_0_1200.png  mnist_0_450.png  mnist_0_750.png
mnist_0_0.png	  mnist_0_150.png   mnist_0_500.png  mnist_0_800.png
mnist_0_1000.png  mnist_0_200.png   mnist_0_50.png   mnist_0_850.png
mnist_0_100.png   mnist_0_250.png   mnist_0_550.png  mnist_0_900.png
mnist_0_1050.png  mnist_0_300.png   mnist_0_600.png  mnist_0_950.png
mnist_0_1100.png  mnist_0_350.png   mnist_0_650.png  sample_data
mnist_0_1150.png  mnist_0_400.png   mnist_0_700.png


In [14]:
files.download('mnist_0_150.png')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
plot_model(combined, show_shapes=False, show_layer_names=True, to_file='gen.png')
files.download('gen.png')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
train(generator, discriminator, combined,
          epochs=1, batch_size=32, save_interval=1)

Number of examples:  60000
Number of Batches:  1875
Number of epochs:  1
Epoch 0 Batch 0/1875 [D loss: 0.848513, acc.: 50.00%] [G loss: 0.884107]


/tensorflow-1.15.2/python3.7/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Epoch 0 Batch 1/1875 [D loss: 0.653577, acc.: 56.25%] [G loss: 0.866115]
Epoch 0 Batch 2/1875 [D loss: 0.629094, acc.: 71.88%] [G loss: 1.478473]
Epoch 0 Batch 3/1875 [D loss: 0.581477, acc.: 75.00%] [G loss: 1.347773]
Epoch 0 Batch 4/1875 [D loss: 0.691689, acc.: 53.12%] [G loss: 1.122229]
Epoch 0 Batch 5/1875 [D loss: 0.568098, acc.: 62.50%] [G loss: 1.412469]
Epoch 0 Batch 6/1875 [D loss: 0.614480, acc.: 59.38%] [G loss: 1.243340]
Epoch 0 Batch 7/1875 [D loss: 0.527628, acc.: 71.88%] [G loss: 1.342524]
Epoch 0 Batch 8/1875 [D loss: 0.597086, acc.: 78.12%] [G loss: 1.253926]
Epoch 0 Batch 9/1875 [D loss: 1.129523, acc.: 31.25%] [G loss: 1.194668]
Epoch 0 Batch 10/1875 [D loss: 0.532330, acc.: 78.12%] [G loss: 0.978352]
Epoch 0 Batch 11/1875 [D loss: 0.803988, acc.: 43.75%] [G loss: 0.965373]
Epoch 0 Batch 12/1875 [D loss: 0.697613, acc.: 56.25%] [G loss: 1.153216]
Epoch 0 Batch 13/1875 [D loss: 0.540083, acc.: 62.50%] [G loss: 1.216839]
Epoch 0 Batch 14/1875 [D loss: 0.759930, acc.: 

...

<IPython.core.display.Javascript object>

Epoch 0 Batch 1201/1875 [D loss: 0.701539, acc.: 59.38%] [G loss: 1.103534]
Epoch 0 Batch 1202/1875 [D loss: 0.633296, acc.: 65.62%] [G loss: 0.998631]
Epoch 0 Batch 1203/1875 [D loss: 0.584177, acc.: 65.62%] [G loss: 1.051771]
Epoch 0 Batch 1204/1875 [D loss: 0.604170, acc.: 62.50%] [G loss: 1.079786]
Epoch 0 Batch 1205/1875 [D loss: 0.625955, acc.: 62.50%] [G loss: 0.956252]
Epoch 0 Batch 1206/1875 [D loss: 0.766899, acc.: 56.25%] [G loss: 0.925524]
Epoch 0 Batch 1207/1875 [D loss: 0.664866, acc.: 62.50%] [G loss: 0.895079]
Epoch 0 Batch 1208/1875 [D loss: 0.668249, acc.: 50.00%] [G loss: 0.959988]
Epoch 0 Batch 1209/1875 [D loss: 0.531683, acc.: 71.88%] [G loss: 0.994594]
Epoch 0 Batch 1210/1875 [D loss: 0.633988, acc.: 53.12%] [G loss: 0.997853]
Epoch 0 Batch 1211/1875 [D loss: 0.741036, acc.: 50.00%] [G loss: 0.933977]
Epoch 0 Batch 1212/1875 [D loss: 0.624539, acc.: 56.25%] [G loss: 0.845489]
Epoch 0 Batch 1213/1875 [D loss: 0.553265, acc.: 78.12%] [G loss: 1.035998]
Epoch 0 Batc

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Epoch 0 Batch 1251/1875 [D loss: 0.765306, acc.: 50.00%] [G loss: 1.031404]
Epoch 0 Batch 1252/1875 [D loss: 0.666394, acc.: 62.50%] [G loss: 1.147850]
Epoch 0 Batch 1253/1875 [D loss: 0.788977, acc.: 37.50%] [G loss: 1.038377]
Epoch 0 Batch 1254/1875 [D loss: 0.614088, acc.: 59.38%] [G loss: 1.123209]
Epoch 0 Batch 1255/1875 [D loss: 0.615969, acc.: 65.62%] [G loss: 1.014430]
Epoch 0 Batch 1256/1875 [D loss: 0.616742, acc.: 62.50%] [G loss: 0.940568]
Epoch 0 Batch 1257/1875 [D loss: 0.548283, acc.: 75.00%] [G loss: 0.979553]
Epoch 0 Batch 1258/1875 [D loss: 0.586234, acc.: 78.12%] [G loss: 1.170407]
Epoch 0 Batch 1259/1875 [D loss: 0.658233, acc.: 65.62%] [G loss: 1.074808]
Epoch 0 Batch 1260/1875 [D loss: 0.803048, acc.: 53.12%] [G loss: 1.055265]
Epoch 0 Batch 1261/1875 [D loss: 0.653558, acc.: 56.25%] [G loss: 1.123082]
Epoch 0 Batch 1262/1875 [D loss: 0.674920, acc.: 56.25%] [G loss: 1.017447]
Epoch 0 Batch 1263/1875 [D loss: 0.693569, acc.: 53.12%] [G loss: 1.019382]
Epoch 0 Batc

In [ ]:
train(generator, discriminator, combined,
          epochs=4, batch_size=32, save_interval=1)

In [ ]:
#endhere

In [ ]:
#def main():
  #  generator, discriminator, combined = build_models()

   # train(generator, discriminator, combined,
      #    epochs=100, batch_size=32, save_interval=1)

In [ ]:
#if __name__ == '__main__':
 #   main()

In [ ]:

##encodermodel = build_generator()
##encodermodel.summary()

##noise = np.random.normal(0, 1, (50, 100))
##fake_images = encodermodel.predict(noise)


In [ ]:
##encodermodel.predict(inputnoise)